# Week 3 Peer Reviewed Project
-------------------------------------------------------------

## Question 1 (Notebook 1)
--------------------------------------

### Creating the Pandas Dataframe from table of postal codes (Wikepedia)

In [85]:
import numpy as np  # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [118]:
CLIENT_ID = 'MZ3JL4T55MTSIFMPDW3324N3T4GZEIYD1D0UI54TICQLTTC0' # your Foursquare ID
CLIENT_SECRET = 'SKADGZAI1CCQHOCDNTTMHFQFWORCLPMKXTPUMKSUBGAQ1UOW' # your Foursquare Secret
VERSION = '20180323'
LIMIT = 30
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MZ3JL4T55MTSIFMPDW3324N3T4GZEIYD1D0UI54TICQLTTC0
CLIENT_SECRET:SKADGZAI1CCQHOCDNTTMHFQFWORCLPMKXTPUMKSUBGAQ1UOW


### Web-Scraping data and reading into a Pandas dataframe

In [18]:
df_canada = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [19]:
# Cleaning up imported dataset Canadian Postal codes data (df_canada)

In [44]:
df_canada.drop(df_canada[df_canada.Borough == 'Not assigned'].index, inplace = True)

df_canada.head()


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Displaying Number of rows in our dataframe df_canada

In [47]:
rows = df_canada.shape[0]
print("The number of rows in our dataframe is" , rows)

The number of rows in our dataframe is 103


## Question 2 (Notebook 2)
---------------------------------------------

### Importing CSV file of Geographical coordiantes of each postal code

In [50]:
location_data = pd.read_csv('http://cocl.us/Geospatial_data')
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Location data with Borough data

In [74]:
canada_full = pd.merge(df_canada, location_data, on= "Postal Code", how = "left")

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(canada_full['Borough'].unique()),
        canada_full.shape[0]
    )
)

canada_full.head(10)

The dataframe has 10 boroughs and 103 neighborhoods.


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Question 3 (Notebook 3)
----------------------

### Creating Toronto Neighborhood Dataset

In [101]:
print(canada_full.Borough.drop_duplicates())      # This was used to see how many unique Toronto neighborhoods
canada = canada_full.copy()      

canada.set_index("Borough", inplace = True)     # Set Boroughs as index values so I can seperate them using loc()

toronto_data = canada.loc[['Downtown Toronto', 'East Toronto', 'West Toronto' , 'Central Toronto']]   # Used loc to pick out Toronto neighborhoods
toronto_data.reset_index(inplace = True)

print('\n\n\nThe new dataframe (toronto_data) now has {} Toronto boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

toronto_data.head()


0           North York
2     Downtown Toronto
5            Etobicoke
6          Scarborough
8            East York
16                York
19        East Toronto
31        West Toronto
61     Central Toronto
76         Mississauga
Name: Borough, dtype: object



The new dataframe (toronto_data) now has 4 Toronto boroughs and 39 neighborhoods.


,Borough,Postal Code,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,M5C,St. James Town,43.651494,-79.375418
4,Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306


### Visualizing using map of Toronto with neighborhoods in all boroughs superimposed on top

In [90]:
latitude = 43.6532
longitude = -79.3832
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
    

### Exploring Neighborhoods in Toronto

In [96]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            43.753259, 
            -79.329656, 
            radius, 
            LIMIT)
url


result111 = requests.get(url).json()
result111

{'meta': {'code': 200, 'requestId': '5f350f6766f09152f168caaa'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7577590045, 'lng': -79.32343773980772},
   'sw': {'lat': 43.7487589955, 'lng': -79.33587426019228}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'contact': {},
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labe

In [119]:

# This Function is used to find details about near by venues in Toronto and saves into a dataframe

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [120]:
# I run the function above and create a new df called venues_toronto

venues_toronto = getNearbyVenues(names= toronto_data['Neighbourhood'],
                                   latitudes= toronto_data['Latitude'],
                                   longitudes= toronto_data['Longitude']
                                  )

# Check how many venues were returned for each neighborhood

venues_toronto.groupby('Neighborhood').count()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, F

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


### Analyze each individual neighborhood

In [122]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_toronto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues_toronto['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0

### Group rows by neighborhood now and take mean of the frequency of every occurence (similar to new york dataset)

In [123]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.033333,0.066667,0.0,0.0,0.033333,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.066667,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.000000,0.0,0.0,0.033333,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0000,0.0,0.033333,0.0,0.000000,0.0,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.033333,0.000000,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0000,0.033333,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.033333,0.0,0.0,0.000000,0.000000,0.0000,0.066667,0.0,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0,0.000000,0.000000,0.0,0.0,0.033333,0.

### Create a new data frame to display top 5 venues for each neighborhood

In [124]:
# This function sort venue values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [126]:
# Creating a new data frame neighborhood_venues_sorted

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Convenience Store
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Park,Pizza Place,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Seafood Restaurant,Sandwich Place
5,Christie,Grocery Store,Café,Park,Coffee Shop,Diner
6,Church and Wellesley,Coffee Shop,Gay Bar,Park,Beer Bar,Café
7,"Commerce Court, Victoria Hotel",Café,Hotel,Japanese Restaurant,Gastropub,Coffee Shop
8,Davisville,Dessert Shop,Coffee Shop,Gym,Pizza Place,Café
9,Davisville North,Food & Drink Shop,Park,Breakfast Spot,Department Store,Gym


## Cluster the Toronto Neighborhoods

### Run k-means clustering and create a new dataframe with a Cluster labels column

In [148]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3], dtype=int32)

In [149]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)
merged_data_toronto = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged_data_toronto = merged_data_toronto.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

merged_data_toronto.head() 

,Borough,Postal Code,Neighbourhood,Latitude,Longitude,Cluster_Labels,Cluster-Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,0,0,0,Coffee Shop,Park,Café,Theater,Breakfast Spot
1,Downtown Toronto,M7A,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0,0,Coffee Shop,Diner,Smoothie Shop,Beer Bar,Distribution Center
2,Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937,0,0,0,Café,Clothing Store,Coffee Shop,Theater,Shopping Mall
3,Downtown Toronto,M5C,St. James Town,43.651494,-79.375418,0,0,0,Gastropub,Café,Coffee Shop,Farmers Market,Ice Cream Shop
4,Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306,0,0,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Farmers Market


### Visualize the neighborhood clusters

In [150]:
# create map

latitude = 43.6532
longitude = -79.3832

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_data_toronto['Latitude'], merged_data_toronto['Longitude'], merged_data_toronto['Neighbourhood'], merged_data_toronto['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## View and Examine each cluster more in depth to find possible differences based on venues, or not.

In [152]:
merged_data_toronto.shape

(39, 13)

### Cluster 1

In [153]:
merged_data_toronto.loc[merged_data_toronto['Cluster_Labels'] == 0, merged_data_toronto.columns[[1] + list(range(5, merged_data_toronto.shape[1]))]]

,Postal Code,Cluster_Labels,Cluster-Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,0,0,0,Coffee Shop,Park,Café,Theater,Breakfast Spot
1,M7A,0,0,0,Coffee Shop,Diner,Smoothie Shop,Beer Bar,Distribution Center
2,M5B,0,0,0,Café,Clothing Store,Coffee Shop,Theater,Shopping Mall
3,M5C,0,0,0,Gastropub,Café,Coffee Shop,Farmers Market,Ice Cream Shop
4,M5E,0,0,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Beer Bar,Farmers Market
5,M5G,0,0,0,Coffee Shop,Café,Italian Restaurant,Seafood Restaurant,Sandwich Place
6,M6G,0,0,0,Grocery Store,Café,Park,Coffee Shop,Diner
7,M5H,0,0,0,Café,Coffee Shop,Hotel,Steakhouse,Vegetarian / Vegan Restaurant
8,M5J,0,0,0,Park,Hotel,Plaza,Café,Roof Deck
9,M5K,0,0,0,Coffee Shop,Café,Japanese Restaurant,Restaurant,Sandwich Place


### Cluster 2

In [154]:
merged_data_toronto.loc[merged_data_toronto['Cluster_Labels'] == 1, merged_data_toronto.columns[[1] + list(range(5, merged_data_toronto.shape[1]))]]

,Postal Code,Cluster_Labels,Cluster-Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4T,1,1,1,Park,Gym,Tennis Court,Wine Shop,Coworking Space


### Cluster 3

In [155]:
merged_data_toronto.loc[merged_data_toronto['Cluster_Labels'] == 2, merged_data_toronto.columns[[1] + list(range(5, merged_data_toronto.shape[1]))]]

,Postal Code,Cluster_Labels,Cluster-Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
30,M4N,2,2,1,Bus Line,Park,Swim School,Wine Shop,Distribution Center


### Cluster 4

In [156]:
merged_data_toronto.loc[merged_data_toronto['Cluster_Labels'] == 3, merged_data_toronto.columns[[1] + list(range(5, merged_data_toronto.shape[1]))]]

,Postal Code,Cluster_Labels,Cluster-Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,M4W,3,3,1,Park,Playground,Trail,Wine Shop,Cosmetics Shop
31,M5N,3,3,2,Garden,Pool,Home Service,Wine Shop,Cosmetics Shop
32,M4P,3,3,0,Food & Drink Shop,Park,Breakfast Spot,Department Store,Gym
33,M5P,3,3,1,Park,Bus Line,Trail,Sushi Restaurant,Jewelry Store
